In [10]:
import datetime
import pymongo
import jieba
import time
import sys

from pymongo.errors import BulkWriteError
from pymongo import MongoClient

In [11]:
fb_DOC = '../data/facebook/'
ettoday_DOC = '../data/ettoday/'
BATCH_SIZE = 4000

In [12]:
client = MongoClient('localhost', 27017)
mydb = client.mydb

In [13]:
news = mydb.news
fb = mydb.facebook

In [33]:
def getInfo(data, num):
    start = 0
    for line in data:
       
        line = line.encode('UTF-8', 'replace')
        line = line.decode('UTF-8')
        line = line.strip()
        
        if num == 1:
            if line.startswith('@GAISRec:'):
                record = dict()
            elif line.startswith('@U:'):
                record['url'] = line[3:]
            elif line.startswith('@T:'):
                record['title'] = line[3:]
            elif line.startswith('@B:'):
                pass
            else:
                line = jieba.cut(line)
                line = " ".join(line)
                #print(line)
                record['content'] = line
                record['from'] = 'ettoday1'
                yield record
                
        if num == 2 :
            if line.startswith('content:') or line.startswith('message'):
                record = dict()
                line = jieba.cut(line[8:])
                line = " ".join(line)
                record['content'] = line
                record['from'] = 'haterccu'
            elif line.startswith('created_time:'):
                record['created_time'] = line[13:];
            elif line.startswith('id:'):
                record['id'] = line[3:]
                record['title'] = '靠北中正-Facebook'    
            elif line.startswith('url:'):
                record['title'] = '靠北中正-Facebook'
                record['url'] = line[4:]
                yield record

        
    
        if num == 3:
            if start is 0:
                start = 1
                continue
            if line.startswith('@Gais_REC:'):
                record = dict()
                body = ''
            elif line.startswith('@url:'):
                record['url'] = line[5:]
            elif line.startswith('@MainTextMD5:'):
                #record['MainTextMD5'] = line[13:]
                continue
            elif line.startswith('@UntagMD5:'):
                #record['UntagMD5'] = line[10:]
                continue
            elif line.startswith('@SiteCode:'):
                #record['SiteCode'] = line[10:]
                continue
            elif line.startswith('@UrlCode:'):
                #record['UrlCode'] = line[9:]
                continue
            elif line.startswith('@title:'):
                record['title'] = line[7:]
            elif line.startswith('@Size:'):
                #record['Size'] = line[6:]
                continue
            elif line.startswith('@keyword:'):
                continue
                record['keyword'] = line[9:].split(',')
            elif line.startswith('@image_links:'):
                continue
                record['image_links'] = line[13:]
            elif line.startswith('@Fetchtime:'):
                continue
            elif line.startswith('@Time:'):
                continue
            elif line.startswith('@post_time:'):
                continue
            elif line.startswith('@Ref:'):
                continue
                record['Ref'] = line[5:]
            elif line.startswith('@BodyMD5:'):
                #record['BodyMD5'] = line[9:]
                continue
            elif line.startswith('@Lang:'):
                continue
                record['Lang'] = line[6:]
            elif line.startswith('@IP:'):
                continue
                record['IP'] = line[4:]

            elif line.startswith('@body:'):
                line = jieba.cut(line[6:])
                #print(len(line))
                line = " ".join(line)
                if len(line) < 2:
                    continue
                #print(len(line), '~~~~'+line)
                body = body + line
            elif line.startswith('@'):
                record['content'] = body
                record['from'] = 'ettoday2'
                yield record
            else:
                line = jieba.cut(line)
                line = " ".join(line)
                if len(line) < 2:
                    continue
                #print(len(line), '****'+line)
                body = body + line

In [34]:
cnt = 530279   #0 or 530279 for old and new datas
doc1 = ['ettoday0.rec', 'ettoday1.rec', 'ettoday2.rec', 'ettoday3.rec', 'ettoday4.rec', 'ettoday5.rec']
doc2 = ['haterccu.rec', 'haterccu1.rec', 'haterccu2.rec']
doc3 = ['ettoday-new']
batchTime = []
batchAvg = 0
batchCnt = 0
start = time.time()
for d in doc3:
    with open(ettoday_DOC+d, "r", encoding='UTF-8', errors="surrogateescape") as fileopen:
        data = fileopen
        acts = []
        for rec in getInfo(data, 3):
            #print(rec)
            #print(cnt)
            rec['_id'] = cnt
            #print('rec _id={0}'.format(rec['_id']))
            #print(rec['content'])
            
            #   use rec.copy() fix the problem BulkWriteError   #
            acts.append(rec.copy())
            
            cnt += 1
            if len(acts) == BATCH_SIZE:
                batch_start = time.time()
                print('finish {0} datas'.format(cnt))
                try:
                    news.insert_many(acts)
                except BulkWriteError as bwe:
                    print(bwe.details)
                    break
                batch_end = time.time()
                batch = batch_end - batch_start
                batchTime.append(batch)
                batchAvg += batch;
                batchCnt += 1;
                print("This batch cost {0}s".format(batch))
                acts = []
        if len(acts) > 0:
            news.insert_many(acts)
            acts = []
        fileopen.close()
end = time.time()
print(cnt)
print("total cost {0}s\nAverage batch cost {1}s\n".format((end - start), batchAvg / batchCnt))

finish 534279 datas
This batch cost 0.2947859764099121s
finish 538279 datas
This batch cost 0.30003952980041504s
finish 542279 datas
This batch cost 0.30849432945251465s
finish 546279 datas
This batch cost 0.28233933448791504s
finish 550279 datas
This batch cost 0.20900297164916992s
finish 554279 datas
This batch cost 0.1952660083770752s
finish 558279 datas
This batch cost 0.3194453716278076s
finish 562279 datas
This batch cost 0.3011949062347412s
finish 566279 datas
This batch cost 0.31101012229919434s
finish 570279 datas
This batch cost 0.23434996604919434s
finish 574279 datas
This batch cost 0.20528054237365723s
finish 578279 datas
This batch cost 0.29355883598327637s
finish 582279 datas
This batch cost 0.3657393455505371s
finish 586279 datas
This batch cost 0.1631166934967041s
finish 590279 datas
This batch cost 0.2583730220794678s
finish 594279 datas
This batch cost 0.2897450923919678s
finish 598279 datas
This batch cost 0.35680150985717773s
finish 602279 datas
This batch cost 0.2

finish 1114279 datas
This batch cost 0.2685863971710205s
finish 1118279 datas
This batch cost 0.3081943988800049s
finish 1122279 datas
This batch cost 0.27705836296081543s
finish 1126279 datas
This batch cost 0.2916073799133301s
finish 1130279 datas
This batch cost 0.27853894233703613s
finish 1134279 datas
This batch cost 0.2058243751525879s
finish 1138279 datas
This batch cost 0.2048490047454834s
finish 1142279 datas
This batch cost 0.3779411315917969s
finish 1146279 datas
This batch cost 0.19899487495422363s
finish 1150279 datas
This batch cost 0.36724305152893066s
finish 1154279 datas
This batch cost 0.35992884635925293s
finish 1158279 datas
This batch cost 0.35040760040283203s
finish 1162279 datas
This batch cost 0.3382859230041504s
finish 1166279 datas
This batch cost 0.27881669998168945s
finish 1170279 datas
This batch cost 0.5399229526519775s
finish 1174279 datas
This batch cost 0.29245853424072266s
finish 1178279 datas
This batch cost 0.3792140483856201s
finish 1182279 datas
Th

finish 1686279 datas
This batch cost 0.2728915214538574s
finish 1690279 datas
This batch cost 0.31456756591796875s
finish 1694279 datas
This batch cost 0.2660329341888428s
finish 1698279 datas
This batch cost 0.31948351860046387s
finish 1702279 datas
This batch cost 0.3124542236328125s
finish 1706279 datas
This batch cost 0.34715723991394043s
finish 1710279 datas
This batch cost 0.31911349296569824s
finish 1714279 datas
This batch cost 0.2840609550476074s
finish 1718279 datas
This batch cost 0.2608511447906494s
finish 1722279 datas
This batch cost 0.2829780578613281s
finish 1726279 datas
This batch cost 0.1877739429473877s
finish 1730279 datas
This batch cost 0.25454020500183105s
finish 1734279 datas
This batch cost 0.17928767204284668s
finish 1738279 datas
This batch cost 0.1681971549987793s
finish 1742279 datas
This batch cost 0.1792607307434082s
finish 1746279 datas
This batch cost 0.2955608367919922s
finish 1750279 datas
This batch cost 0.1867678165435791s
finish 1754279 datas
This

finish 2258279 datas
This batch cost 0.3092508316040039s
finish 2262279 datas
This batch cost 0.18878746032714844s
finish 2266279 datas
This batch cost 0.3088533878326416s
finish 2270279 datas
This batch cost 0.34940171241760254s
finish 2274279 datas
This batch cost 0.3655109405517578s
finish 2278279 datas
This batch cost 0.32245540618896484s
finish 2282279 datas
This batch cost 0.2824532985687256s
finish 2286279 datas
This batch cost 0.40926289558410645s
finish 2290279 datas
This batch cost 0.19129085540771484s
finish 2294279 datas
This batch cost 0.32494354248046875s
finish 2298279 datas
This batch cost 0.2748730182647705s
finish 2302279 datas
This batch cost 0.3221440315246582s
finish 2306279 datas
This batch cost 0.28420472145080566s
finish 2310279 datas
This batch cost 0.3347587585449219s
finish 2314279 datas
This batch cost 0.31404972076416016s
finish 2318279 datas
This batch cost 0.3487355709075928s
finish 2322279 datas
This batch cost 0.31842541694641113s
finish 2326279 datas
T

finish 2830279 datas
This batch cost 0.23864054679870605s
finish 2834279 datas
This batch cost 0.2800304889678955s
finish 2838279 datas
This batch cost 0.3237440586090088s
finish 2842279 datas
This batch cost 0.2088162899017334s
finish 2846279 datas
This batch cost 0.32153940200805664s
finish 2850279 datas
This batch cost 0.19155097007751465s
finish 2854279 datas
This batch cost 0.3139815330505371s
finish 2858279 datas
This batch cost 0.25521278381347656s
finish 2862279 datas
This batch cost 0.3296215534210205s
finish 2866279 datas
This batch cost 0.3124120235443115s
finish 2870279 datas
This batch cost 0.2661919593811035s
finish 2874279 datas
This batch cost 0.3112308979034424s
finish 2878279 datas
This batch cost 0.27639079093933105s
finish 2882279 datas
This batch cost 0.2898671627044678s
finish 2886279 datas
This batch cost 0.1922600269317627s
finish 2890279 datas
This batch cost 0.3205687999725342s
finish 2894279 datas
This batch cost 0.3096182346343994s
finish 2898279 datas
This 

finish 3402279 datas
This batch cost 0.216292142868042s
finish 3406279 datas
This batch cost 0.3434717655181885s
finish 3410279 datas
This batch cost 0.9149658679962158s
finish 3414279 datas
This batch cost 0.22434592247009277s
finish 3418279 datas
This batch cost 0.22483110427856445s
finish 3422279 datas
This batch cost 0.3817477226257324s
finish 3426279 datas
This batch cost 0.28790736198425293s
finish 3430279 datas
This batch cost 0.2523939609527588s
finish 3434279 datas
This batch cost 0.5672776699066162s
finish 3438279 datas
This batch cost 0.2956054210662842s
finish 3442279 datas
This batch cost 0.2830533981323242s
finish 3446279 datas
This batch cost 0.32724881172180176s
finish 3450279 datas
This batch cost 0.3438572883605957s
finish 3454279 datas
This batch cost 0.262615442276001s
finish 3458279 datas
This batch cost 0.156203031539917s
finish 3462279 datas
This batch cost 0.3083038330078125s
finish 3466279 datas
This batch cost 0.24968576431274414s
finish 3470279 datas
This bat

finish 3974279 datas
This batch cost 0.2773900032043457s
finish 3978279 datas
This batch cost 0.19730854034423828s
finish 3982279 datas
This batch cost 0.37767624855041504s
finish 3986279 datas
This batch cost 0.3534226417541504s
finish 3990279 datas
This batch cost 0.32075071334838867s
finish 3994279 datas
This batch cost 0.31293749809265137s
finish 3998279 datas
This batch cost 0.3256957530975342s
finish 4002279 datas
This batch cost 0.21852493286132812s
finish 4006279 datas
This batch cost 0.246567964553833s
finish 4010279 datas
This batch cost 0.1834404468536377s
finish 4014279 datas
This batch cost 0.25742483139038086s
finish 4018279 datas
This batch cost 0.19426512718200684s
finish 4022279 datas
This batch cost 0.28798723220825195s
finish 4026279 datas
This batch cost 0.2638983726501465s
finish 4030279 datas
This batch cost 0.4160459041595459s
finish 4034279 datas
This batch cost 0.2875385284423828s
finish 4038279 datas
This batch cost 0.30770230293273926s
finish 4042279 datas
Th

finish 4546279 datas
This batch cost 0.22204327583312988s
finish 4550279 datas
This batch cost 0.2920339107513428s
finish 4554279 datas
This batch cost 0.2817261219024658s
finish 4558279 datas
This batch cost 0.36090564727783203s
finish 4562279 datas
This batch cost 0.2034451961517334s
finish 4566279 datas
This batch cost 0.48036670684814453s
finish 4570279 datas
This batch cost 0.3619251251220703s
finish 4574279 datas
This batch cost 0.3043346405029297s
finish 4578279 datas
This batch cost 0.3846879005432129s
finish 4582279 datas
This batch cost 0.2392120361328125s
finish 4586279 datas
This batch cost 0.1472337245941162s
finish 4590279 datas
This batch cost 0.4174830913543701s
finish 4594279 datas
This batch cost 0.3260512351989746s
finish 4598279 datas
This batch cost 0.5524694919586182s
finish 4602279 datas
This batch cost 0.2928152084350586s
finish 4606279 datas
This batch cost 0.34400153160095215s
finish 4610279 datas
This batch cost 0.25827646255493164s
finish 4614279 datas
This 

finish 5118279 datas
This batch cost 0.5531163215637207s
finish 5122279 datas
This batch cost 0.3251771926879883s
finish 5126279 datas
This batch cost 0.2777590751647949s
finish 5130279 datas
This batch cost 0.29804396629333496s
finish 5134279 datas
This batch cost 0.2704024314880371s
finish 5138279 datas
This batch cost 0.33428525924682617s
finish 5142279 datas
This batch cost 0.31302738189697266s
finish 5146279 datas
This batch cost 0.20378446578979492s
finish 5150279 datas
This batch cost 0.30693864822387695s
finish 5154279 datas
This batch cost 0.26566600799560547s
finish 5158279 datas
This batch cost 0.31859827041625977s
finish 5162279 datas
This batch cost 0.30170559883117676s
finish 5166279 datas
This batch cost 0.2945430278778076s
finish 5170279 datas
This batch cost 0.21454811096191406s
finish 5174279 datas
This batch cost 0.3092529773712158s
finish 5178279 datas
This batch cost 0.22793126106262207s
finish 5182279 datas
This batch cost 0.4278411865234375s
finish 5186279 datas


In [13]:
print(batchTime)
print(batchCnt)


[0.4333992004394531, 0.397611141204834, 0.4349091053009033, 0.32015109062194824, 0.29435276985168457, 0.3091573715209961, 0.2797262668609619, 0.4181098937988281, 0.27834057807922363, 0.3129720687866211, 0.3140230178833008, 0.2885556221008301, 0.3662381172180176, 0.2708742618560791, 0.2002732753753662, 0.2904477119445801, 0.2916390895843506, 0.2695455551147461, 0.2902095317840576, 0.3846583366394043, 0.3365483283996582, 0.19979476928710938, 0.30342626571655273, 0.28395652770996094, 0.28632593154907227, 0.1992948055267334, 0.20539021492004395, 0.3234431743621826, 0.3494391441345215, 0.35215139389038086, 0.318042516708374, 0.2430877685546875, 0.32353830337524414, 0.270308256149292, 0.7039501667022705, 0.28933215141296387, 0.28903818130493164, 0.2977292537689209, 0.363067626953125, 0.26108717918395996, 0.25244712829589844, 0.29945945739746094, 0.31269359588623047, 0.20087432861328125, 0.35988378524780273, 0.5377500057220459, 0.31595826148986816, 0.32169604301452637, 0.23686504364013672, 0.